In [1]:
import sys
import os
sys.path.append('../') # root directory

In [2]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 2.3 MB/s eta 0:00:01
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [3]:
from constants import constant
import pandas as pd
import numpy as np
import constants.constant as constant
import utils.commonutils as utils
import utils.scraperutils as scraperutils
from constants.namedtuples import Column
from write.JsonFileWriter import write
from read.JsonFileReader import parse
from pathlib import Path

In [10]:
SP500_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
NASDAQ100_URL = 'https://en.wikipedia.org/wiki/NASDAQ-100'
OUT_DIR = Path('../output')
IN_DIR = Path('../input')

In [5]:
def extract_wiki_table(table_soup):
    '''
    quirks -> all 'th'.stripped_strings[0] are columns, tr.stripped_strings gives row data
    '''
    th_list = table_soup.find_all('th')
    columns = [list(th.stripped_strings)[0] for th in th_list] #stripped[0] because of \n in some
    print(f'Colums:\n{columns}')
    tbody = table_soup.find('tbody')
    rows = tbody.find_all('tr')
    data = [dict(zip(columns, list(row.stripped_strings))) for row in rows[1:]]
    return data

# SP500

In [13]:
soup = scraperutils.get_beautiful_soup(SP500_URL)
table_soup = soup.find("table", id="constituents")
sp500 = extract_wiki_table(table_soup)
sp500_file = f'SP500_Wiki_{utils.current_datetime_str()}'
write(OUT_DIR/sp500_file, sp500)
pd.DataFrame(parse(OUT_DIR/sp500_file)).head()

Colums:
['Symbol', 'Security', 'SEC filings', 'GICS', 'GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK', 'Founded']


,Symbol,Security,SEC filings,GICS,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981


# NASDAQ100

In [7]:
soup = scraperutils.get_beautiful_soup(NASDAQ100_URL)
table_soup = soup.find("table", id="constituents")
nasdaq100 = extract_wiki_table(table_soup)
nasdaq100_file = f'NASDAQ100_Wiki_{utils.current_datetime_str()}'
write(OUT_DIR/nasdaq100_file, nasdaq100)
nasdaq100 = pd.DataFrame(parse(OUT_DIR/nasdaq100_file))

Colums:
['Company', 'Ticker', 'GICS', 'GICS']


,Company,Ticker,GICS
0,Activision Blizzard,ATVI,Interactive Home Entertainment
1,Adobe,ADBE,Application Software
2,ADP,ADP,Data Processing & Outsourced Services
3,Airbnb,ABNB,Internet & Direct Marketing Retail
4,Align,ALGN,Health Care Supplies
5,Alphabet,(Class A),Communication Services
6,Alphabet,(Class C),Communication Services
7,Amazon,AMZN,Internet & Direct Marketing Retail
8,AMD,AMD,Semiconductors
9,American Electric Power,AEP,Electric Utilities


In [59]:
pd.DataFrame(nasdaq100).to_csv(OUT_DIR/f'NASDAQ100.csv')
# pd.DataFrame(sp500).to_csv(OUT_DIR/f'SP500.csv')

# Write to StockList

In [60]:
# nasdaq100 = pd.DataFrame(nasdaq100)
# sp500 = pd.DataFrame(sp500)
stocklist = pd.concat([nasdaq100['Ticker'], sp500['Symbol']])
stocklist = pd.DataFrame({'Symbol': stocklist})
stocklist[~stocklist.duplicated()].to_excel(IN_DIR/'StockList.xlsx', index=False)

In [63]:
pd.read_excel(IN_DIR/'StockList.xlsx')

,Symbol
0,ATVI
1,ADBE
2,ADP
3,ABNB
4,ALGN
...,...
524,YUM
525,ZBRA
526,ZBH
527,ZION


In [58]:
nasdaq100.loc[nasdaq100['Ticker'].str.contains('Class C'), 'Ticker'] = 'GOOG'

In [62]:
stocklist

,Symbol
0,ATVI
1,ADBE
2,ADP
3,ABNB
4,ALGN
...,...
501,YUM
502,ZBRA
503,ZBH
504,ZION
